Robotic Systems 1 - Homework 2

In [ ]:
import numpy as np # Linear Algebra
import pinocchio as pin # Pinocchio library
import os

from pinocchio.robot_wrapper import RobotWrapper
from pinocchio.visualize import MeshcatVisualizer

VISUALIZER = MeshcatVisualizer

In [ ]:
def load_franka():
  current_path = os.path.abspath('') # where the folder `robot` is located at
  robot_path = os.path.join(current_path, "robot")

  robot = RobotWrapper.BuildFromURDF(os.path.join(robot_path, "franka.urdf"), package_dirs = robot_path)

  return robot.model

In [ ]:
model = load_franka()
data = model.createData()



In [ ]:
#cur_state: [q,v]'
def step_world(model, data, cur_state, control, dt):
  model.nq
